<h1 align="center">
 Download dos Metadados do Catálogo de Teses e Dissertações <br/>
 <img src="https://dadosabertos.capes.gov.br/img/caixa.png"  alt="Dados Capes"/>
</h1>

No portal de dados abertos da *CAPES*, há dados sobre os procedimentos realizados e informações sobre os programas de pós-graduação do Brasil. Parte dos dados são fornecidos por coordenadores desses programas através do módulo **Coleta CAPES** disponível na [Plataforma Sucupira](https://sucupira.capes.gov.br/sucupira/).

Dentre esses dados, estamos interessados no **Catálogo de Teses e Dissertações** que contém informações sobre as teses e dissertações defendidas nos programas de pós-graduação. No portal, há três conjuntos que contemplam informações sobre essas produções dividida em períodos de tempos sendo eles: 

- [[1987 a 2012] Catálogo de Teses e Dissertações - Brasil](https://dadosabertos.capes.gov.br/dataset/1987-a-2012-catalogo-de-teses-e-dissertacoes-brasil)
- [[2013 a 2016] Catálogo de Teses e Dissertações - Brasil](https://dadosabertos.capes.gov.br/dataset/catalogo-de-teses-e-dissertacoes-de-2013-a-2016)
- [[2017 a 2019] Catálogo de Teses e Dissertações - Brasil](https://dadosabertos.capes.gov.br/dataset/2018-catalogo-de-teses-e-dissertacoes-da-capes)

Nosso objetivo é realizar o download dessas tabelas, unir e padronizar as colunas de dados em comum. 

## Acessando os dados

Para obter a lista de conjuntos do portal de dados abertos da **CAPES** relacionada ao catálogo de teses e dissertações, podemos consultar a API do CKAN filtrando pelo termo de busca `catalogo-de-teses-e-dissertacoes`:

```
curl https://dadosabertos.capes.gov.br/api/3/action/package_search?q=catalogo-de-teses-e-dissertacoes
```
Nessa consulta obtivemos um json contendo 3 resultados representando os conjuntos de teses e dissertações disponíveis no website.

Para compreender mais como usar o *CKAN API*, acesse o [API guide](https://docs.ckan.org/en/2.9/api/index.html).

In [1]:
# bibliotecas utilizadas
import shutil 
import requests
import json
import os
import pandas as pd

# diretório onde será salvo os dados
workdir = "catalogo-de-teses-e-dissertacoes"

# cria pastas que conterão os arquivos CSVs
os.makedirs(workdir, exist_ok=True)

### Download dos dados

Para acessar a resposta da _CKAN API_ utilizamos a biblioteca `requests` e extraímos a variável _result_.

In [ ]:
# conjunto de dados que queremos
payload = {"q": "catalogo-de-teses-e-dissertacoes"}

# realizar a requisição HTTP
response = requests.get(
    "https://dadosabertos.capes.gov.br/api/3/action/package_search?",
    params=payload
)

# se a requisiçao foi realizada com sucesso (status code = 200)
assert response.status_code == requests.codes.ok

# atribuir json da resposta a um dicionário
response_dict = response.json()

# checar o conteúdo da resposta
assert response_dict["success"] is True

# obter conteúdo retornado
result = response_dict["result"]

No _json_ retornado, estamos interessados na variável _results_ que contém o link para *download* dos recursos. Os dados estão disponíveis em dois formatos CSV e XLS, os metadados estão em PDF e HTML. Desejamos baixar apenas os arquivos CSV, então filtramos por essa informação. 

In [ ]:
filtered_resources = {}
for item in result["results"]: 
  # obtém apenas os recursos com dado no formato csv
  resources_csv = [r for r in item["resources"] if r["format"] == "CSV"] 
  # guarda os recursos de cada conjunto
  filtered_resources[item["name"]] = resources_csv

print(filtered_resources.keys())

# salvar dados sobre os recursos no disco
with open(f"{workdir}/resources.json", "w") as f:
  json.dump(filtered_resources, f, indent=4)

Após ter informações sobre os dados baixados podemos os guardar no disco.

In [ ]:
# cria pasta para cada conjunto de resultados com base no nome de cada um
for current_dir in filtered_resources.keys():
  os.makedirs(f"{workdir}/{current_dir}", exist_ok=True)
  print(f"Criado o diretório {current_dir}")

# percorre os dados a serem salvos no disco
for key, items in filtered_resources.items():
  for item in items:
    filename = os.path.basename(item["url"]) # define nome do arquivo
    download = requests.get(item["url"], stream=True)
    # salva arquivo na pasta designada
    with open(f"{workdir}/{key}/{filename}", "w") as output_file:
      # escreve 10000 linhas de cada vez
      for chunk in download.iter_content(chunk_size=10000, decode_unicode=True):
        output_file.write(chunk)
      output_file.flush()

### Agrupar dados por conjunto

Há três grupos de dados de teses e dissertações como mencionamos no começo desse notebook. Ao analisarmos os metadados contido no portal de dados abertos observamos que os conjuntos de 2013 a 2016 e 2017 a 2019 possuem o mesmo tipo de dados. Com base nisso, vamos agrupar os arquivos CSVs em dois grandes arquivos:

- `catalogo-de-teses-e-dissertacoes-de-1987-a-2012.csv` - contempla todos os dados coletados no período de 1987 a 2012
- `catalogo-de-teses-e-dissertacoes-de-2013-a-2019.csv` - contempla todos os dados coletados no período de 2013 a 2019

O ano base corresponde ao ano que foi realizada a coleta, acessando o portal de dados abertos de um desses dados podemos observar que cada csv é equivalente a um ano:

- BR-CAPES-BTD-2017-2018-08-01_2017
- BR-CAPES-BTD-2018-2019-09-09_2018
- BR-CAPES-BTD-2019-2020-11-20_2019

Com o objetivo de preencher os valores faltantes que podem ocorrer nessa coluna usamos a operação `Dataframe.interpolate()`.

In [ ]:
def merge_files(file_path, output_path):
  list_df = []
  for path in file_path:
    for filename in os.listdir(path):
      # carregar arquivo
      data = pd.read_csv(os.path.join(path, filename), sep=";")
      # dados antes de 2013 coluna se chamana AnoBase
      if "AnoBase" in data:
        data["ano_base"] = data["AnoBase"]
      # dados a partir de 2013 coluna se chama AN_BASE
      if "AN_BASE" in data:
        data["ano_base"] = data["AN_BASE"]
      # preenche os nans via interpolação linear
      data["ano_base"] = data["ano_base"].interpolate()
      # adicionar dados ao conjunto
      list_df.append(data)
  df = pd.concat(list_df, ignore_index=True)
  # salvar arquivo no disco
  df.to_csv(output_path, index=False)
  return df

In [ ]:
between_2013_2019 = [
              f"{workdir}/catalogo-de-teses-e-dissertacoes-de-2013-a-2016", 
              f"{workdir}/2018-catalogo-de-teses-e-dissertacoes-da-capes"
]

before_2013 = [f"{workdir}/1987-a-2012-catalogo-de-teses-e-dissertacoes-brasil"]

In [ ]:
before_2013 =  merge_files(before_2013, f"{workdir}/catalogo-de-teses-e-dissertacoes-de-1987-a-2012.csv")
before_2013.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (29,30,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (29,30,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (29,30,31,32) have mix

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676338 entries, 0 to 676337
Data columns (total 42 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   AnoBase                  676338 non-null  int64 
 1   CodigoPrograma           676338 non-null  object
 2   Regiao                   676338 non-null  object
 3   Uf                       676338 non-null  object
 4   SiglaIes                 676338 non-null  object
 5   NomeIes                  676338 non-null  object
 6   NomePrograma             676338 non-null  object
 7   GrandeAreaCodigo         676338 non-null  int64 
 8   GrandeAreaDescricao      676338 non-null  object
 9   AreaConhecimentoCodigo   676338 non-null  int64 
 10  AreaConhecimento         676338 non-null  object
 11  AreaAvaliacao            676338 non-null  object
 12  DocumentoDiscente        676338 non-null  object
 13  Autor                    676338 non-null  object
 14  TituloTese          

In [ ]:
from_2013 = merge_files(between_2013_2019, f"{workdir}/catalogo-de-teses-e-dissertacoes-de-2013-a-2019.csv")
from_2013.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (17,19) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 559458 entries, 0 to 559457
Data columns (total 57 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   AN_BASE                      559456 non-null  float64
 1   CD_PROGRAMA                  559458 non-null  object 
 2   NM_PROGRAMA                  559457 non-null  object 
 3   SG_ENTIDADE_ENSINO           559457 non-null  object 
 4   NM_ENTIDADE_ENSINO           559457 non-null  object 
 5   ID_ADD_PRODUCAO_INTELECTUAL  559457 non-null  float64
 6   ID_PRODUCAO_INTELECTUAL      559457 non-null  float64
 7   NM_PRODUCAO                  559457 non-null  object 
 8   ID_SUBTIPO_PRODUCAO          559457 non-null  float64
 9   NM_SUBTIPO_PRODUCAO          559457 non-null  object 
 10  ID_AREA_CONCENTRACAO         539093 non-null  float64
 11  NM_AREA_CONCENTRACAO         559457 non-null  object 
 12  ID_LINHA_PESQUISA            508465 non-null  float64
 13 

## Padronização de colunas

As informações armazenadas antes de 2013 e a partir de 2013 sobre as teses e dissertações possuem padrões de colunas diferentes, vamos selecionar apenas as colunas em comum e renomeá-las para possuírem os mesmos nomes. Com base nas informações nos metadados definimos as seguintes informações em comum:

- cod_programa
- regiao
- sigla_uf
- sigla_entidade_ensino
- nome_entidade_ensino
- nome_programa
- cod_grande_area
- nome_grande_area
- cod_area_conhecimento
- nome_area_conhecimento
- nome_area_avaliacao
- nome_discente
- titulo_producao
- grau_academico
- palavra_chave
- nr_volume
- nr_paginas
- idioma
- resumo
- linha_pesquisa
- url_texto_completo

In [2]:
selected_columns = [
"ano_base",
"titulo_producao",
"resumo",
"palavra_chave",
"idioma",
"grau_academico",
"cod_grande_area",
"nome_grande_area",
"cod_area_conhecimento",
"nome_area_conhecimento",
"cod_programa",
"nome_programa",
"sigla_entidade_ensino",
"nome_entidade_ensino",
"nome_area_avaliacao",
"linha_pesquisa",
"nr_volume",
"nr_paginas",
"regiao",
"sigla_uf",
"url_texto_completo",
"nome_discente",
]

In [3]:
before_2013.rename({
    "CodigoPrograma": "cod_programa",
    "Regiao": "regiao",
    "Uf": "sigla_uf",
    "SiglaIes": "sigla_entidade_ensino",
    "NomeIes": "nome_entidade_ensino",
    "NomePrograma": "nome_programa",
    "GrandeAreaCodigo" : "cod_grande_area",
    "GrandeAreaDescricao" :"nome_grande_area",
    "AreaConhecimentoCodigo":"cod_area_conhecimento",
    "AreaConhecimento":"nome_area_conhecimento",
    "AreaAvaliacao": "nome_area_avaliacao",
    "Autor": "nome_discente",
    "TituloTese": "titulo_producao",
    "Nivel": "grau_academico",
    "PalavrasChave": "palavra_chave",
    "Volume": "nr_volume",
    "NumeroPaginas": "nr_paginas",
    "Idioma": "idioma",
    "ResumoTese": "resumo",
    "LinhaPesquisa": "linha_pesquisa",
    "URLTextoCompleto": "url_texto_completo"
},axis=1, inplace=True)

before_2013 = before_2013[selected_columns]
before_2013.loc[:,"idioma":"sigla_uf"] = before_2013.loc[:,"idioma":"sigla_uf"] .apply(
    lambda x: x.str.upper() if(x.dtype == 'object') else x
)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (24,27,28,29,30,31,32,35,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
from_2013.rename({
    "CD_PROGRAMA": "cod_programa",
    "NM_REGIAO": "regiao",
    "SG_UF_IES": "sigla_uf",
    "SG_ENTIDADE_ENSINO": "sigla_entidade_ensino",
    "NM_ENTIDADE_ENSINO": "nome_entidade_ensino",
    "NM_PROGRAMA": "nome_programa",
    "CD_GRANDE_AREA_CONHECIMENTO" : "cod_grande_area",
    "NM_GRANDE_AREA_CONHECIMENTO" :"nome_grande_area",
    "CD_AREA_CONHECIMENTO":"cod_area_conhecimento",
    "NM_AREA_CONHECIMENTO":"nome_area_conhecimento",
    "NM_AREA_AVALIACAO": "nome_area_avaliacao",
    "NM_DISCENTE": "nome_discente",
    "NM_PRODUCAO": "titulo_producao",
    "NM_GRAU_ACADEMICO": "grau_academico",
    "DS_PALAVRA_CHAVE": "palavra_chave",
    "NR_VOLUME": "nr_volume",
    "NR_PAGINAS": "nr_paginas",
    "NM_IDIOMA": "idioma",
    "DS_RESUMO": "resumo",
    "NM_LINHA_PESQUISA": "linha_pesquisa",
    "DS_URL_TEXTO_COMPLETO": "url_texto_completo"
},axis=1, inplace=True)


from_2013 = from_2013[selected_columns]

from_2013.loc[:,"idioma":"sigla_uf"] = from_2013.loc[:,"idioma":"sigla_uf"] .apply(
    lambda x: x.str.upper() if(x.dtype == 'object') else x
)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (45,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Agrupando conjunto contendo informações de todas as teses e dissertações

Por fim, realizamos o merge de ambos os conjuntos de dados em um grande CSV que poderá ser utilizado em análise futuras.

In [5]:
# agrupar dados e ordenar pelo ano de coleta
final_data = pd.concat([before_2013, from_2013], ignore_index=True)
final_data = final_data.sort_values("ano_base")

final_data.to_csv(
    f"{workdir}/catalogo-de-teses-e-dissertacoes.csv",
    index=False
)

In [6]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235796 entries, 675551 to 1235795
Data columns (total 22 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1235796 non-null  float64
 1   titulo_producao         1235734 non-null  object 
 2   resumo                  1235571 non-null  object 
 3   palavra_chave           1234724 non-null  object 
 4   idioma                  1165930 non-null  object 
 5   grau_academico          1235795 non-null  object 
 6   cod_grande_area         1235795 non-null  float64
 7   nome_grande_area        1235795 non-null  object 
 8   cod_area_conhecimento   1235795 non-null  float64
 9   nome_area_conhecimento  1235795 non-null  object 
 10  cod_programa            1235796 non-null  object 
 11  nome_programa           1235795 non-null  object 
 12  sigla_entidade_ensino   1235795 non-null  object 
 13  nome_entidade_ensino    1235795 non-null  object 
 1

In [7]:
#@title Pasta do Google Drive 
#@markdown Inclua o caminho do diretório do drive onde você deseja salvar os dados.
destination = "/content/drive/MyDrive/AcademicAI/" #@param {type:"string"}

if os.path.exists(destination):
  # Move o diretório atual para o informado
  dest = shutil.move(workdir, destination) 